In [5]:
import pymongo
import base64
import numpy as np
from PIL import Image
from io import BytesIO

# Verbindung zur MongoDB herstellen
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Daten abrufen
data = collection.find()

# Canvas-Bilder dekodieren und in numpy Arrays umwandeln
def decode_canvas(canvas_data):
    image_data = base64.b64decode(canvas_data.split(",")[1])
    image = Image.open(BytesIO(image_data))
    return np.array(image)

# Zielgröße für alle Bilder festlegen (z.B. 35x280)
target_size = (35, 280)

# Funktion zum Resizing der Bilder
def resize_image(image, target_size):
    return np.array(Image.fromarray(image).resize(target_size, Image.ANTIALIAS))

# Alle Canvas-Bilder dekodieren und auf die Zielgröße bringen
canvases = []
labels = []
for record in data:
    for canvas in record["canvases"]:
        decoded_image = decode_canvas(canvas)
        resized_image = resize_image(decoded_image, target_size)
        canvases.append(resized_image)
        labels.append(record["username"])

canvases = np.array(canvases)
labels = np.array(labels)


c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Bilder skalieren
canvases = canvases / 255.0

# Labels in numerische Werte umwandeln
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(canvases, labels, test_size=0.2, random_state=42)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Modell erstellen
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(35, 280, 4)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Modell kompilieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modell zusammenfassen
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 33, 278, 32)       1184      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 139, 32)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 137, 64)       18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 68, 64)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 30464)             0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [10]:
# Modell trainieren
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10


ValueError: in user code:

    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\input

In [ ]:
# Modell evaluieren
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")


ValueError: in user code:

    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\input_s

In [ ]:
# Modell speichern
model.save("canvas_auth_model.h5")
